<a href="https://colab.research.google.com/github/jamespan1118/NeuralData/blob/main/PointNet%2B%2B_familiarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install tensorboard --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 5.2 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.11.0 which is incompatible.


In [3]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import plotly.graph_objects as go
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

###Data loading

In [4]:
class NeuralData(Dataset):
  def __init__(self,data,transform=None):
     
     self.x = data[:,0]
     self.y = data[:,1]
     self.n_samples = data.shape[0]
     self.transform = transform

  def __getitem__(self,index):
    sample = self.x[index],self.y[index]
    if self.transform:
      sample = self.transform(sample)
    return sample

  def __len__(self):
    return self.n_samples

# Downsample function for each spike sequence
class Downsample:
  def __init__(self,samplesize):
    self.samplesize=samplesize
  
  def __call__(self,sample):
    features,label = sample
    idx = np.random.randint(len(features), size=self.samplesize)
    return features[idx,:],label

# Return only three elements
class Pickthree:
  def __call__(self,sample):
    features,label = sample
    return features[:,1:],label

composed = torchvision.transforms.Compose([
    Downsample(2000),
    Pickthree()
])

In [5]:
data = np.load('/content/drive/MyDrive/modelnet/stacked.npy',allow_pickle=True)
len_data = data.shape[0]
train_idx, val_idx = train_test_split([i for i in range(len_data)],test_size=0.2)

train_dataset = NeuralData(data=data[train_idx, :],transform=composed)
val_dataset = NeuralData(data=data[val_idx, :], transform=composed)

In [6]:
len(train_idx) / 32

7.125

In [7]:
train_dataloader = DataLoader(dataset=train_dataset,batch_size=16,shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset,batch_size=16,shuffle=False)

In [8]:
#testing that dataloader works
train_dataiter = iter(val_dataloader)
data = train_dataiter.next()
features,label = data
print(features,label)

tensor([[[3.1914e+01, 3.7659e+03, 1.3009e+01],
         [1.1912e+01, 1.6535e+03, 2.1710e+01],
         [2.8916e+01, 4.4254e+02, 1.0574e+01],
         ...,
         [4.2182e+01, 1.2919e+03, 8.5609e+00],
         [1.1457e+01, 7.6587e+02, 8.7563e+00],
         [5.2416e+01, 1.8249e+03, 1.4805e+01]],

        [[2.0703e+01, 8.1886e+02, 1.9517e+01],
         [5.2303e+01, 1.6442e+03, 9.2769e+00],
         [7.1448e+01, 1.2453e+03, 7.8879e+00],
         ...,
         [1.9253e+01, 3.1349e+03, 8.6443e+00],
         [6.8752e+01, 2.0829e+02, 1.1152e+01],
         [3.5730e+00, 1.0321e+03, 6.9389e+00]],

        [[8.1421e+01, 2.9198e+02, 7.4585e+00],
         [2.7154e+01, 9.7906e+02, 9.4324e+00],
         [6.8837e+01, 2.6173e+02, 8.9561e+00],
         ...,
         [1.0505e+01, 3.1472e+03, 1.0998e+01],
         [7.7641e+00, 3.0804e+03, 1.2786e+01],
         [1.5899e+01, 5.9690e+02, 1.2098e+01]],

        ...,

        [[5.2434e+01, 9.4622e+02, 2.3428e+01],
         [5.8265e+01, 3.0721e+03, 1.0270e+01]

### Model

In [9]:
class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix


class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

In [10]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    #print(outputs.device, labels.device)
    return criterion(outputs, labels.type(torch.LongTensor).to(device)) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

###Training

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
pointnet = PointNet()
pointnet.to(device);

In [13]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

In [14]:
len(train_dataset)

228

In [28]:
def train(model, train_loader, val_loader=None,  epochs=15, save=True):
    writer = SummaryWriter()
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        sum_loss = []
        sum_loss_valid = []

        for i, data in enumerate(train_loader, 0):
            inputs = data[0]
            labels = data[1]
            #print(type(inputs), type(labels))
            inputs = inputs.float()
            labels = labels
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            sum_loss.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss = loss.item()
            print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                        (epoch + 1, i + 1, len(train_loader), running_loss / 10))
            
        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data[0].to(device).float(), data[1].to(device)
                    
                    outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))
                    val_loss = pointnetloss(outputs, labels, m3x3, m64x64)
                    sum_loss_valid.append(val_loss)

                    running_loss = val_loss.item()
                    
                    print('[Epoch: %d, Batch: %4d / %4d], val_loss: %.3f' %
                        (epoch + 1, i + 1, len(train_loader), running_loss / 10))

                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        if save:
            torch.save(pointnet.state_dict(), "save_"+str(epoch)+".pth")
        
        writer.add_scalars("Loss",{"Train":sum(sum_loss)/len(sum_loss),"Validation":sum(sum_loss_valid)/len(sum_loss_valid)},epoch+1)
    writer.flush()

In [29]:
train(pointnet, train_dataloader, val_dataloader,  save=False)

[Epoch: 1, Batch:    1 /   15], loss: 0.050
[Epoch: 1, Batch:    2 /   15], loss: 0.071
[Epoch: 1, Batch:    3 /   15], loss: 0.055
[Epoch: 1, Batch:    4 /   15], loss: 0.067
[Epoch: 1, Batch:    5 /   15], loss: 0.043
[Epoch: 1, Batch:    6 /   15], loss: 0.061
[Epoch: 1, Batch:    7 /   15], loss: 0.052
[Epoch: 1, Batch:    8 /   15], loss: 0.068
[Epoch: 1, Batch:    9 /   15], loss: 0.077
[Epoch: 1, Batch:   10 /   15], loss: 0.045
[Epoch: 1, Batch:   11 /   15], loss: 0.059
[Epoch: 1, Batch:   12 /   15], loss: 0.067
[Epoch: 1, Batch:   13 /   15], loss: 0.062
[Epoch: 1, Batch:   14 /   15], loss: 0.039
[Epoch: 1, Batch:   15 /   15], loss: 0.062
[Epoch: 1, Batch:   15 /   15], val_loss: 0.033
[Epoch: 1, Batch:   15 /   15], val_loss: 0.059
[Epoch: 1, Batch:   15 /   15], val_loss: 0.045
[Epoch: 1, Batch:   15 /   15], val_loss: 0.039
Valid accuracy: 86 %
[Epoch: 2, Batch:    1 /   15], loss: 0.065
[Epoch: 2, Batch:    2 /   15], loss: 0.065
[Epoch: 2, Batch:    3 /   15], loss: 0

In [30]:
! tensorboard dev upload --logdir runs --name "My latest experiment" --description "Simple comparison of several hyperparameters" # optional

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/DK0gbm6mRGWNJjAcPXf1bg/

[2022-11-29T03:46:12] Started scanning logdir.
[2022-11-29T03:46:12] Total uploaded: 30 scalars, 0 tensors, 0 binary objects


Interrupted. View your TensorBoard at https://tensorboard.dev/experiment/DK0gbm6mRGWNJjAcPXf1bg/
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/pro

In [ ]:
#for visualization of point cloud data
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [ ]:
x = features[:,1]
z = features[:,2]
ptp = features[:,3]
visualize_rotate([go.Mesh3d(x=x, y=ptp, z=z, color='lightpink', opacity=0.50)]).show()